In [1]:
import pandas as pd

import gym
import numpy as np
from stable_baselines3 import A2C, DDPG, SAC, PPO
from matplotlib import axis
import numpy as np
from floris.tools import FlorisInterface
from floris.tools.visualization import visualize_cut_plane
import matplotlib.pyplot as plt
from floris.tools.optimization.yaw_optimization.yaw_optimizer_sr import YawOptimizationSR

#Importing custom environment.
from Custom_functions import CustomEnv, CustomEnv2


In [4]:
df = pd.read_excel (r'Results\Overview.xlsx')

In [5]:
model_compare = "A2C3"

In [6]:
index = df[df["Model name"]==model_compare].index.values[0]

In [7]:
Model_name = df["Model name"][index]
t_steps = df["t_steps"][index]
dist = df["dist"][index]
nx = df["nx"][index]
ny = df["ny"][index]
turb_type = df["turb_type"][index]
combination = df["combination"][index]
deflection = df["deflection"][index]
turbulence = df["turbulence"][index]
velocity = df["velocity"][index]
WS_min = df["WS_min"][index]
WS_max = df["WS_max"][index]
TI_min = df["TI_min"][index]
TI_max = df["TI_max"][index]
wd_min = df["wd_min"][index]
wd_max = df["wd_max"][index]
yaw_max = df["yaw_max"][index]
rho = df["rho"][index]

In [8]:
env = CustomEnv(t_steps, dist, nx, ny, turb_type, combination, deflection,
               turbulence, velocity, WS_min, WS_max, TI_min, TI_max, wd_min, wd_max, 
               yaw_max, rho)

In [9]:
model_path = "models/"+model_compare+"/475000"

In [10]:
model = A2C.load(model_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
#Creates the base for the farm
fi = FlorisInterface("gch.yaml")   

#Turns it into a dictionary and then does the changes to the model
fi_dict = fi.floris.as_dict()

fi_dict["farm"]["turbine_type"] = [turb_type]
fi_dict["wake"]["model_strings"]["combination_model"] = combination
fi_dict["wake"]["model_strings"]["deflection_model"]  = deflection
fi_dict["wake"]["model_strings"]["turbulence_model"]  = turbulence
fi_dict["wake"]["model_strings"]["velocity_model"]    = velocity
fi_dict["flow_field"]["air_density"]                  = rho

# Turns it back into a floris object:
fi = FlorisInterface(fi_dict)

D = fi.floris.farm.rotor_diameters[0]

x = np.linspace(0, D*dist*nx, nx)
y = np.linspace(0, D*dist*ny, ny)

xv, yv = np.meshgrid(x, y, indexing='xy')

layout_x =  xv.flatten()
layout_y = yv.flatten()

fi.reinitialize(
            layout=(layout_x, layout_y),
            )


In [12]:
TI = TI_min

actions = []
agent_power = []
baseline_power = []
wind_dirs = []
yaw_angles = np.zeros((1, 1, nx*ny)) 
yaw_zero = np.zeros((1, 1, nx*ny)) 

wind_speed = 7



ws_norm = (wind_speed - env.wind_speed_min)/(env.wind_speed_max - env.wind_speed_min)
TI_norm = (TI - env.TI_min)/(env.TI_max - env.TI_min)  




for wind_dir in range(wd_min, wd_max+1):
    wind_dirs.append(wind_dir)
    wd_norm = (wind_dir - env.wd_min)/(env.wd_max - env.wd_min)
    #observation = np.array([self.wd_norm, self.ws_norm, self.TI_norm], dtype = np.float32)
    observation = np.array([wd_norm, ws_norm, TI_norm], dtype = np.float32)

    action, _states = model.predict(observation, deterministic=True)
    
    action = action*env.yaw_max   #scales the action to the 
    actions.append(action)
    
    fi.reinitialize(
        wind_directions=[wind_dir],
        turbulence_intensity= TI,
        wind_speeds=[wind_speed]
        )
    
    yaw_angles[0,0,:] = action
    fi.calculate_wake(yaw_angles = yaw_zero)
    
    baseline_power.append(fi.get_farm_power()[0][0])
    
    fi.calculate_wake(yaw_angles = yaw_angles)
    agent_power.append(fi.get_farm_power()[0][0])
        
        

actions = np.array(actions)
np.unique(actions,axis = 0)

baseline_power = np.array(baseline_power)
agent_power = np.array(agent_power)
wind_dirs = np.array(wind_dirs)


In [13]:
fig = plt.gcf()
fig.set_size_inches(15/1.5, 7/1.5)
ax = plt.gca()
plt.plot(agent_power,wind_dirs, label = '1')
#plt.plot(step2,rew2, label = '2')
#plt.xlabel("Episode number", fontsize = 12, fontweight="bold")
#plt.ylabel("Average reward", fontsize = 12, fontweight="bold")
#plt.legend(title="Agent number", fontsize = 12)
#plt.title("The reward as a function of the episode number", fontsize = 12, fontweight="bold")
#plt.grid()
#plt.xlim(0,510_000)
#plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
#plt.show()
#fig.savefig('Plots/Fluid meetings/rew_eps1.png', dpi=150)

: 

: 

In [131]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [132]:
import tensorflow as tf
import datetime, os

In [133]:
%tensorboard --logdir logs